# Data

## season_game_level_data

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col
from pylab import hist, show
import scipy
import zipfile


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

**data frames used in this notebook:**
- da = pbp
- dg = season_games_data
- dm = play_by_play
- dp = player_rankings
- dw = team_roster_player_rank
- dv = season_team_roster_ranking
- dx = season_game_team_roster
- dz = season_team
- dy = season_game_roster

**for analysis:**
- ds = season_level
- dl = season_game_level
- dt = season_game_team_level

In [2]:
pwd

'/Users/stefanostselios/Desktop/nhl_roster_design-master'

### import play by play data set

In [3]:
da = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/pbp_merged.csv')
#da = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/pbp_merged.csv')
da = da.drop('Unnamed: 0', axis=1)
da = da.rename(columns={'TeamCode': 'EventTeamCode'})


keep regular season games

In [4]:
da = da[da['GameNumber'] <= 21230]

drop irrelevant data

In [5]:
da = da[da['EventType']!='STOP']
da = da[da['EventType']!='EISTR']
da = da[da['EventType']!='EIEND']

- display goals for each game and drop duplicates.

In [6]:
dh = da[da['EventTeamCode']==da['HTeamCode']]
dh = dh.rename(columns={'EventTeamCode': 'HTeam'})
dh['goal'] = dh.apply(lambda x: 1 if (x['EventType'] == 'GOAL') else 0, axis=1)
dh['HGF'] = dh.groupby(['Season', 'GameNumber', 'HTeam'])['goal'].transform('sum')
dh = dh[['Season', 'GameNumber', 'HGF']]
dh = dh.drop_duplicates(['Season', 'GameNumber'])

In [7]:
dv = da[da['EventTeamCode']==da['VTeamCode']]
dv = dv.rename(columns={'EventTeamCode': 'VTeam'})
dv['goal'] = dv.apply(lambda x: 1 if (x['EventType'] == 'GOAL') else 0, axis=1)
dv['VGF'] = dv.groupby(['Season', 'GameNumber', 'VTeam'])['goal'].transform('sum')
dv = dv[['Season', 'GameNumber', 'VGF']]
dv = dv.drop_duplicates(['Season', 'GameNumber'])

Merge into season-game data

In [8]:
dg = da[['Season', 'GameNumber', 'VTeamCode', 'HTeamCode']]
dg = dg.drop_duplicates(['Season', 'GameNumber'])

In [9]:
dg = pd.merge(dg, dh, on=['Season', 'GameNumber'], how='left')
dg = pd.merge(dg, dv, on=['Season', 'GameNumber'], how='left')

- find the goal differential per game with respect to home team.

In [10]:
dg['GD'] = dg['HGF'] - dg['VGF']
dg['WinTeam'] = dg.apply(lambda x: x['HTeamCode'] if x['GD'] > 0 else x['VTeamCode'], axis=1)
dg['LossTeam'] = dg.apply(lambda x: x['HTeamCode'] if x['WinTeam'] != x['HTeamCode'] else x['VTeamCode'], axis=1)

In [11]:
dg.head()

,Season,GameNumber,VTeamCode,HTeamCode,HGF,VGF,GD,WinTeam,LossTeam
0,2010,20001,MTL,TOR,3,2,1,TOR,MTL
1,2010,20002,PHI,PIT,2,3,-1,PHI,PIT
2,2010,20003,CAR,MIN,3,4,-1,CAR,MIN
3,2010,20004,CHI,COL,4,3,1,COL,CHI
4,2010,20005,CGY,EDM,4,0,4,EDM,CGY


- display goals againest per team.

In [12]:
dg['VGA'] = dg['HGF']
dg['HGA'] = dg['VGF']

In [13]:
dg = dg[['Season', 'GameNumber', 'VTeamCode', 'HTeamCode', 'VGF', 'HGF', 'GD', 'VGA', 'HGA', 'WinTeam', 'LossTeam']]
dg.head()

,Season,GameNumber,VTeamCode,HTeamCode,VGF,HGF,GD,VGA,HGA,WinTeam,LossTeam
0,2010,20001,MTL,TOR,2,3,1,3,2,TOR,MTL
1,2010,20002,PHI,PIT,3,2,-1,2,3,PHI,PIT
2,2010,20003,CAR,MIN,4,3,-1,3,4,CAR,MIN
3,2010,20004,CHI,COL,3,4,1,4,3,COL,CHI
4,2010,20005,CGY,EDM,0,4,4,4,0,EDM,CGY


In [14]:
dg.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/season_game_data.csv', index='False', sep=',')
#dg.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/season_game_data.csv', index='False', sep=',')

## season_level_data

In [15]:
dm = da.copy()

events that happened in regulation time only

In [16]:
dm = dm[dm['Period'] <= 3]
dm = dm[dm['Period'] >= 1]

- **reshape the data set from wide to long.**

In [17]:
dm = dm.sort_values(['Season', 'GameNumber', 'Period', 'EventNumber'], ascending=[True, True, True, True])

In [18]:
a = [col for col in dm.columns if 'VPlayer' in col]
b = [col for col in dm.columns if 'HPlayer' in col]
c = [col for col in dm.columns if 'VPosition' in col]
d = [col for col in dm.columns if 'HPosition' in col]
dm = pd.lreshape(dm, {'VPlayer' : a, 'HPlayer' : b, 'VPosition' : c, 'HPosition': d})

In [19]:
dm.shape

(1796745, 24)

In [20]:
dm = dm.rename(columns={'PlayerNumber': 'EventPlayerNumber', 'TeamCode': 'EventTeamCode', 'PlayerName': 'EventPlayerName' })
dm = dm[['Season', 'GameNumber', 'GameDate', 'Period', 'AdvantageType', 'Zone', 'EventNumber', 'EventType', 'EventDetail', 'EventTeamCode', 'EventPlayerNumber', 'EventPlayerName', 'EventTimeFromZero', 'EventTimeFromTwenty', 'VTeamCode', 'VPlayer', 'VPosition', 'HTeamCode', 'HPlayer', 'HPosition', 'ShotType', 'ShotResult', 'Length', 'PenaltyType']]
dm = dm.sort_values(['Season', 'GameNumber', 'Period', 'EventNumber'], ascending=[True, True, True, True])

- fill in advantage type with even strength 'EV' and event player number with 'TEAM'

In [21]:
dm['AdvantageType'] = dm['AdvantageType'].fillna('EV')
dm['EventPlayerNumber'] = dm['EventPlayerNumber'].fillna('TEAM')

- save new datast as play by play

In [22]:
dm.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/play_by_play.csv', index='False', sep=',')
#dm.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/play_by_play.csv', index='False', sep=',')

#### create new data set and keep variables: 
- (a) game number.
- (b) visitor team information.
- (c) home team information.

In [23]:
df = dm.copy()
df = df[['Season', 'GameNumber', 'VTeamCode', 'VPlayer', 'VPosition', 'HTeamCode', 'HPlayer', 'HPosition']]
df = df.sort_values(['Season', 'GameNumber'], ascending=[True, True])
df.head()

,Season,GameNumber,VTeamCode,VPlayer,VPosition,HTeamCode,HPlayer,HPosition
0,2010,20001,MTL,11.0,C,TOR,37.0,C
310113,2010,20001,MTL,21.0,R,TOR,9.0,R
620126,2010,20001,MTL,57.0,L,TOR,11.0,L
930061,2010,20001,MTL,26.0,D,TOR,3.0,D
1239931,2010,20001,MTL,75.0,D,TOR,22.0,D


- merge season_game_data (dg) on new dataset

In [24]:
df = pd.merge(df, dg, on=['Season', 'GameNumber', 'VTeamCode', 'HTeamCode'], how='left')
df.head()    

,Season,GameNumber,VTeamCode,VPlayer,VPosition,HTeamCode,HPlayer,HPosition,VGF,HGF,GD,VGA,HGA,WinTeam,LossTeam
0,2010,20001,MTL,11.0,C,TOR,37.0,C,2,3,1,3,2,TOR,MTL
1,2010,20001,MTL,21.0,R,TOR,9.0,R,2,3,1,3,2,TOR,MTL
2,2010,20001,MTL,57.0,L,TOR,11.0,L,2,3,1,3,2,TOR,MTL
3,2010,20001,MTL,26.0,D,TOR,3.0,D,2,3,1,3,2,TOR,MTL
4,2010,20001,MTL,75.0,D,TOR,22.0,D,2,3,1,3,2,TOR,MTL


- reshape the data to have home and visitor team observatons under the same coloumns. 

In [25]:
a = [col for col in df.columns if 'Player' in col]
b = [col for col in df.columns if 'Position' in col]
c = [col for col in df.columns if 'TeamCode' in col]
d = [col for col in df.columns if 'GF' in col]
e = [col for col in df.columns if 'GA' in col]
df = pd.lreshape(df, {'PlayerNumber' : a, 'PlayerPosition' : b, 'TeamCode' : c, 'GF' : d, 'GA' : e })
df = df[['Season', 'GameNumber', 'TeamCode', 'PlayerNumber', 'GF', 'GA', 'GD', 'WinTeam', 'LossTeam']]
df.head()

,Season,GameNumber,TeamCode,PlayerNumber,GF,GA,GD,WinTeam,LossTeam
0,2010,20001,MTL,11.0,2,3,1,TOR,MTL
1,2010,20001,MTL,21.0,2,3,1,TOR,MTL
2,2010,20001,MTL,57.0,2,3,1,TOR,MTL
3,2010,20001,MTL,26.0,2,3,1,TOR,MTL
4,2010,20001,MTL,75.0,2,3,1,TOR,MTL


### import player position and rankings

In [26]:
dp = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/clusters/stats_per_player_nhl_positions.csv')
#dp = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/clusters/stats_per_player_nhl_positions.csv')
dp = dp.drop('Unnamed: 0', axis=1)
dp = dp.rename(columns={'Position': 'PlayerPosition'})

In [27]:
dp = dp[['Season', 'TeamCode', 'PlayerNumber', 'PlayerName', 'PlayerPosition']]
dp.head()

,Season,TeamCode,PlayerNumber,PlayerName,PlayerPosition
0,2010,MTL,11.0,GOMEZ,C
1,2010,MTL,21.0,GIONTA,W
2,2010,MTL,57.0,POULIOT,W
3,2010,MTL,26.0,GORGES,D
4,2010,MTL,75.0,GILL,D


In [28]:
dp.shape

(1071, 5)

In [29]:
dr = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/clusters/all_player_rank_nhl_positions.csv')
#dr = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/clusters/all_player_rank_nhl_positions.csv')
dr = dr.drop('Unnamed: 0', axis=1)
dr = dr.rename(columns={'Position': 'PlayerPosition'})

In [30]:
dr = dr[['Season', 'TeamCode', 'PlayerNumber', 'PlayerName', 'PlayerPosition', 'Rank']]
dr = dr.sort_values(['TeamCode'], ascending=[True])
dr.head()

,Season,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank
164,2010,ANA,22.0,MARCHANT,C,3
234,2010,ANA,54.0,FOWLER,D,1
470,2010,ANA,21.0,BROOKBANK,D,3
237,2010,ANA,4.0,FOWLER,D,1
329,2010,ANA,23.0,BEAUCHEMIN,D,2


- merge player position and player rankings and drop goaltenders

In [31]:
ds = pd.merge(dp, dr, on=['Season', 'TeamCode', 'PlayerNumber', 'PlayerName', 'PlayerPosition'], how='left')
ds['Rank'] = ds.groupby(['Season', 'TeamCode', 'PlayerName'])['Rank'].apply(lambda x: x.ffill().bfill())
ds.shape

(1071, 6)

In [32]:
ds.isnull().sum()

Season              0
TeamCode            0
PlayerNumber        0
PlayerName          0
PlayerPosition      0
Rank              241
dtype: int64

skaters that have played less than 9 games were not included in the clusters and therefore not ranked. Since the failed to make the roster on a regular basis, forwards are assigned to the 4th line and defensemen to the bottom (3rd) pairing.

In [33]:
ds['Rank'] = ds.apply(lambda x: 1 if ((x['PlayerPosition'] == 'G') & (pd.isnull(x.Rank))) else 3 if ((x['PlayerPosition'] == 'D') & (pd.isnull(x.Rank))) else 4 if ((x['PlayerPosition'] == 'W') & (pd.isnull(x.Rank))) else 4 if ((x['PlayerPosition'] == 'C') & (pd.isnull(x.Rank))) else x['Rank'], axis=1)
ds = ds.sort_values(['TeamCode'], ascending=[True])
ds.shape

(1071, 6)

In [34]:
ds1 = ds[(ds['PlayerPosition'] == 'D') & (ds['Rank'] == 4)]
ds1.head()

,Season,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank
1008,2010,OTT,51.0,SMITH,D,4.0


- Derek Smith, a defenseman for the Ottawa Senators has a ranking of 4, which is incorrect since we have 3 defensive pairings. For that reason, he is assigned a rank of 3 which represents the bottom defensive pairing

In [35]:
ds['Rank'] = ds.apply(lambda x: 3 if ((x['PlayerPosition'] == 'D') & (x['TeamCode'] == 'OTT') & (x['PlayerName'] == 'SMITH') & (x['PlayerNumber'] == 51.0)) else x['Rank'], axis=1)

In [36]:
ds1 = ds[(ds['PlayerPosition'] == 'D') & (ds['Rank'] == 4)]
ds1.head()

,Season,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank


- **display each player by team per game. Drop duplicates.**

In [41]:
dw = pd.merge(df, ds, on=['Season', 'TeamCode', 'PlayerNumber'], how='left')
dw = dw[['Season', 'GameNumber', 'TeamCode', 'PlayerNumber', 'PlayerName', 'PlayerPosition', 'Rank', 'GF', 'GA', 'GD', 'WinTeam', 'LossTeam']]
dw.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank,GF,GA,GD,WinTeam,LossTeam
0,2010,20001,MTL,11.0,GOMEZ,C,2.0,2,3,1,TOR,MTL
1,2010,20001,MTL,21.0,GIONTA,W,2.0,2,3,1,TOR,MTL
2,2010,20001,MTL,57.0,POULIOT,W,2.0,2,3,1,TOR,MTL
3,2010,20001,MTL,26.0,GORGES,D,2.0,2,3,1,TOR,MTL
4,2010,20001,MTL,75.0,GILL,D,2.0,2,3,1,TOR,MTL


- drop observations that have no player name, position nor ranking. Exclude goaltenders from the roster.

In [42]:
dw = dw[pd.notnull(dw['PlayerPosition'])]
dw.isnull().sum()

Season            0
GameNumber        0
TeamCode          0
PlayerNumber      0
PlayerName        0
PlayerPosition    0
Rank              0
GF                0
GA                0
GD                0
WinTeam           0
LossTeam          0
dtype: int64

In [43]:
dw.shape

(3703346, 12)

- create column that displays the position and roster count by team per game. To simplify matters, we categorize skaters into forwards and defensemen.

In [44]:
dw = dw.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerNumber'])
dw['RosterCount'] = dw.groupby(['Season', 'GameNumber', 'TeamCode'])['PlayerNumber'].transform('count')
dw['PositionCount'] = dw.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerPosition'])['PlayerNumber'].transform('count')
dw.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank,GF,GA,GD,WinTeam,LossTeam,RosterCount,PositionCount
0,2010,20001,MTL,11.0,GOMEZ,C,2.0,2,3,1,TOR,MTL,19.0,7.0
1,2010,20001,MTL,21.0,GIONTA,W,2.0,2,3,1,TOR,MTL,19.0,5.0
2,2010,20001,MTL,57.0,POULIOT,W,2.0,2,3,1,TOR,MTL,19.0,5.0
3,2010,20001,MTL,26.0,GORGES,D,2.0,2,3,1,TOR,MTL,19.0,6.0
4,2010,20001,MTL,75.0,GILL,D,2.0,2,3,1,TOR,MTL,19.0,6.0


- count the amount of forwards and defensemen by team per game.

In [45]:
dw['CCount'] = dw.apply(lambda x: x['PositionCount'] if x['PlayerPosition']=='C' else np.NaN, 1)
dw['WCount'] = dw.apply(lambda x: x['PositionCount'] if x['PlayerPosition']=='W' else np.NaN, 1)
dw['DCount'] = dw.apply(lambda x: x['PositionCount'] if x['PlayerPosition']=='D' else np.NaN, 1)
dw['GCount'] = dw.apply(lambda x: x['PositionCount'] if x['PlayerPosition']=='G' else np.NaN, 1)

dw['CCount'] = dw.groupby(['Season','GameNumber', 'TeamCode'])['CCount'].apply(lambda x: x.ffill().bfill())
dw['WCount'] = dw.groupby(['Season','GameNumber', 'TeamCode'])['WCount'].apply(lambda x: x.ffill().bfill())
dw['DCount'] = dw.groupby(['Season','GameNumber', 'TeamCode'])['DCount'].apply(lambda x: x.ffill().bfill())
dw['GCount'] = dw.groupby(['Season','GameNumber', 'TeamCode'])['GCount'].apply(lambda x: x.ffill().bfill())
dw.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank,GF,GA,GD,WinTeam,LossTeam,RosterCount,PositionCount,CCount,WCount,DCount,GCount
0,2010,20001,MTL,11.0,GOMEZ,C,2.0,2,3,1,TOR,MTL,19.0,7.0,7.0,5.0,6.0,1.0
1,2010,20001,MTL,21.0,GIONTA,W,2.0,2,3,1,TOR,MTL,19.0,5.0,7.0,5.0,6.0,1.0
2,2010,20001,MTL,57.0,POULIOT,W,2.0,2,3,1,TOR,MTL,19.0,5.0,7.0,5.0,6.0,1.0
3,2010,20001,MTL,26.0,GORGES,D,2.0,2,3,1,TOR,MTL,19.0,6.0,7.0,5.0,6.0,1.0
4,2010,20001,MTL,75.0,GILL,D,2.0,2,3,1,TOR,MTL,19.0,6.0,7.0,5.0,6.0,1.0


In [46]:
dw.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/full_season_team_game_player_rank_all_nhl_positions.csv', index='False', sep=',')
#dw.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/full_season_team_game_player_rank_all_nhl_positions.csv', index='False', sep=',')

In [47]:
dw1 = dw.drop_duplicates(['Season', 'GameNumber', 'TeamCode'])
dw1 = dw1.sort_values(['Season', 'GameNumber'], ascending=[True, True])

In [48]:
dw.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank,GF,GA,GD,WinTeam,LossTeam,RosterCount,PositionCount,CCount,WCount,DCount,GCount
0,2010,20001,MTL,11.0,GOMEZ,C,2.0,2,3,1,TOR,MTL,19.0,7.0,7.0,5.0,6.0,1.0
1,2010,20001,MTL,21.0,GIONTA,W,2.0,2,3,1,TOR,MTL,19.0,5.0,7.0,5.0,6.0,1.0
2,2010,20001,MTL,57.0,POULIOT,W,2.0,2,3,1,TOR,MTL,19.0,5.0,7.0,5.0,6.0,1.0
3,2010,20001,MTL,26.0,GORGES,D,2.0,2,3,1,TOR,MTL,19.0,6.0,7.0,5.0,6.0,1.0
4,2010,20001,MTL,75.0,GILL,D,2.0,2,3,1,TOR,MTL,19.0,6.0,7.0,5.0,6.0,1.0


In [49]:
dw1.shape

(2460, 18)

In [50]:
dw1.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank,GF,GA,GD,WinTeam,LossTeam,RosterCount,PositionCount,CCount,WCount,DCount,GCount
0,2010,20001,MTL,11.0,GOMEZ,C,2.0,2,3,1,TOR,MTL,19.0,7.0,7.0,5.0,6.0,1.0
1852892,2010,20001,TOR,37.0,BRENT,C,3.0,3,2,1,TOR,MTL,19.0,5.0,5.0,7.0,6.0,1.0
1611,2010,20002,PHI,17.0,CARTER,C,1.0,3,2,-1,PHI,PIT,19.0,5.0,5.0,7.0,6.0,1.0
1854537,2010,20002,PIT,71.0,MALKIN,C,1.0,2,3,-1,PHI,PIT,19.0,8.0,8.0,4.0,6.0,1.0
3224,2010,20003,CAR,53.0,SKINNER,C,1.0,4,3,-1,CAR,MIN,19.0,6.0,6.0,6.0,6.0,1.0


In [51]:
dw1.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/team_roster_player_rank_all_nhl_positions.csv', index='False', sep=',')
#dw1.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/team_roster_player_rank_all_nhl_positions.csv', index='False', sep=',')

In [52]:
dw1.shape

(2460, 18)

### full regular season stats

In [53]:
dw2 = dw.copy()
dw2 = dw2.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerPosition', 'GF', 'GA', 'RosterCount', 'PositionCount', 'CCount', 'WCount', 'DCount', 'WinTeam', 'LossTeam'], as_index=False)['Rank'].mean()
dw2.shape

(9840, 14)

- create columns for team win and team loss. 

In [54]:
dw2['TeamWin'] =  dw2.apply(lambda x: 1 if x['TeamCode']==x['WinTeam'] else 0, 1)
dw2['TeamLos'] =  dw2.apply(lambda x: 1 if x['TeamCode']==x['LossTeam'] else 0, 1)

- display games played, games won, games loss, goals for and goals against by team for the season.

In [55]:
dw2['GP'] = dw2.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GameNumber'].transform('count')
dw2['GW'] = dw2.groupby(['Season', 'PlayerPosition', 'TeamCode'])['TeamWin'].transform('sum')
dw2['GL'] = dw2.groupby(['Season', 'PlayerPosition', 'TeamCode'])['TeamLos'].transform('sum')
dw2['GF'] = dw2.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GF'].transform('sum')
dw2['GA'] = dw2.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GA'].transform('sum')
dw2.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL
0,2010,20001,MTL,C,220,213,19.0,7.0,7.0,5.0,6.0,TOR,MTL,2.714286,0,1,82,44,38
1,2010,20001,MTL,D,220,213,19.0,6.0,7.0,5.0,6.0,TOR,MTL,1.666667,0,1,82,44,38
2,2010,20001,MTL,G,220,213,19.0,1.0,7.0,5.0,6.0,TOR,MTL,1.000000,0,1,82,44,38
3,2010,20001,MTL,W,220,213,19.0,5.0,7.0,5.0,6.0,TOR,MTL,2.000000,0,1,82,44,38
4,2010,20001,TOR,C,225,259,19.0,5.0,5.0,7.0,6.0,TOR,MTL,2.600000,1,0,82,37,45


- create columns with the mean ranking for forward and defenseman by team per game.

In [56]:
dw2['RankC'] = dw2.apply(lambda x: x['Rank'] if x['PlayerPosition']=='C' else np.NaN, 1)
dw2['RankW'] = dw2.apply(lambda x: x['Rank'] if x['PlayerPosition']=='W' else np.NaN, 1)
dw2['RankD'] = dw2.apply(lambda x: x['Rank'] if x['PlayerPosition']=='D' else np.NaN, 1)
dw2['RankG'] = dw2.apply(lambda x: x['Rank'] if x['PlayerPosition']=='G' else np.NaN, 1)

dw2['RankC'] = dw2.groupby(['Season','GameNumber', 'TeamCode'])['RankC'].apply(lambda x: x.ffill().bfill())
dw2['RankW'] = dw2.groupby(['Season','GameNumber', 'TeamCode'])['RankW'].apply(lambda x: x.ffill().bfill())
dw2['RankD'] = dw2.groupby(['Season','GameNumber', 'TeamCode'])['RankD'].apply(lambda x: x.ffill().bfill())
dw2['RankG'] = dw2.groupby(['Season','GameNumber', 'TeamCode'])['RankG'].apply(lambda x: x.ffill().bfill())
dw2.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,RankC,RankW,RankD,RankG
0,2010,20001,MTL,C,220,213,19.0,7.0,7.0,5.0,6.0,TOR,MTL,2.714286,0,1,82,44,38,2.714286,2.000000,1.666667,1.0
1,2010,20001,MTL,D,220,213,19.0,6.0,7.0,5.0,6.0,TOR,MTL,1.666667,0,1,82,44,38,2.714286,2.000000,1.666667,1.0
2,2010,20001,MTL,G,220,213,19.0,1.0,7.0,5.0,6.0,TOR,MTL,1.000000,0,1,82,44,38,2.714286,2.000000,1.666667,1.0
3,2010,20001,MTL,W,220,213,19.0,5.0,7.0,5.0,6.0,TOR,MTL,2.000000,0,1,82,44,38,2.714286,2.000000,1.666667,1.0
4,2010,20001,TOR,C,225,259,19.0,5.0,5.0,7.0,6.0,TOR,MTL,2.600000,1,0,82,37,45,2.600000,2.285714,2.000000,3.0


- compute the mean per position by team for the season.

In [57]:
dw2['MeanC']= dw2.groupby(['Season', 'TeamCode'])['RankC'].transform('mean')
dw2['MeanW']= dw2.groupby(['Season', 'TeamCode'])['RankW'].transform('mean')
dw2['MeanD']= dw2.groupby(['Season', 'TeamCode'])['RankD'].transform('mean')
dw2['MeanG']= dw2.groupby(['Season', 'TeamCode'])['RankG'].transform('mean')
dw2.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,RankC,RankW,RankD,RankG,MeanC,MeanW,MeanD,MeanG
0,2010,20001,MTL,C,220,213,19.0,7.0,7.0,5.0,6.0,TOR,MTL,2.714286,0,1,82,44,38,2.714286,2.000000,1.666667,1.0,2.341057,1.858420,1.728513,1.158537
1,2010,20001,MTL,D,220,213,19.0,6.0,7.0,5.0,6.0,TOR,MTL,1.666667,0,1,82,44,38,2.714286,2.000000,1.666667,1.0,2.341057,1.858420,1.728513,1.158537
2,2010,20001,MTL,G,220,213,19.0,1.0,7.0,5.0,6.0,TOR,MTL,1.000000,0,1,82,44,38,2.714286,2.000000,1.666667,1.0,2.341057,1.858420,1.728513,1.158537
3,2010,20001,MTL,W,220,213,19.0,5.0,7.0,5.0,6.0,TOR,MTL,2.000000,0,1,82,44,38,2.714286,2.000000,1.666667,1.0,2.341057,1.858420,1.728513,1.158537
4,2010,20001,TOR,C,225,259,19.0,5.0,5.0,7.0,6.0,TOR,MTL,2.600000,1,0,82,37,45,2.600000,2.285714,2.000000,3.0,2.417683,2.291347,2.256678,2.182927


- display the quantity of wins and losses per team for the whole season

In [58]:
dw2['L'] = dw2.apply(lambda x: x['GL'] if x['TeamCode']== x['LossTeam'] else (x['GP'] - x['GW']), 1)
dw2['W'] = dw2.apply(lambda x: x['GW'] if x['TeamCode']== x['WinTeam'] else (x['GP'] - x['GL']), 1)
dw2.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,RankC,RankW,RankD,RankG,MeanC,MeanW,MeanD,MeanG,L,W
0,2010,20001,MTL,C,220,213,19.0,7.0,7.0,5.0,6.0,TOR,MTL,2.714286,0,1,82,44,38,2.714286,2.000000,1.666667,1.0,2.341057,1.858420,1.728513,1.158537,38,44
1,2010,20001,MTL,D,220,213,19.0,6.0,7.0,5.0,6.0,TOR,MTL,1.666667,0,1,82,44,38,2.714286,2.000000,1.666667,1.0,2.341057,1.858420,1.728513,1.158537,38,44
2,2010,20001,MTL,G,220,213,19.0,1.0,7.0,5.0,6.0,TOR,MTL,1.000000,0,1,82,44,38,2.714286,2.000000,1.666667,1.0,2.341057,1.858420,1.728513,1.158537,38,44
3,2010,20001,MTL,W,220,213,19.0,5.0,7.0,5.0,6.0,TOR,MTL,2.000000,0,1,82,44,38,2.714286,2.000000,1.666667,1.0,2.341057,1.858420,1.728513,1.158537,38,44
4,2010,20001,TOR,C,225,259,19.0,5.0,5.0,7.0,6.0,TOR,MTL,2.600000,1,0,82,37,45,2.600000,2.285714,2.000000,3.0,2.417683,2.291347,2.256678,2.182927,45,37


- compute win and loss percent by team. Drop duplicate observations.

In [59]:
dw2 = dw2[['Season', 'TeamCode', 'GP', 'L', 'W', 'GF', 'GA', 'MeanC', 'MeanW', 'MeanD', 'MeanG']]
dw2 = dw2.drop_duplicates(['Season', 'TeamCode'])
dw2['WinPc'] = dw2['W']/ dw2['GP']
dw2['LossPc'] = dw2['L']/ dw2['GP']

dw2 = dw2[['Season', 'TeamCode', 'GP','W', 'L', 'GF', 'GA', 'WinPc', 'LossPc', 'MeanC', 'MeanW', 'MeanD', 'MeanG']]
dw2.head()

,Season,TeamCode,GP,W,L,GF,GA,WinPc,LossPc,MeanC,MeanW,MeanD,MeanG
0,2010,MTL,82,44,38,220,213,0.536585,0.463415,2.341057,1.858420,1.728513,1.158537
4,2010,TOR,82,37,45,225,259,0.451220,0.548780,2.417683,2.291347,2.256678,2.182927
8,2010,PHI,82,47,35,266,233,0.573171,0.426829,1.803659,1.982578,1.613821,1.987805
12,2010,PIT,82,49,33,244,203,0.597561,0.402439,2.103600,2.246051,1.926713,1.756098
16,2010,CAR,82,40,42,239,242,0.487805,0.512195,2.174390,2.246951,1.847561,1.243902


- rank teams based on win percent, mean centres, wingers and defensemen. 

In [60]:
dw2['RankWin'] = dw2.groupby(['Season'])['WinPc'].rank(ascending=False)
dw2['RankC'] = dw2.groupby(['Season'])['MeanC'].rank(ascending=True)
dw2['RankW'] = dw2.groupby(['Season'])['MeanW'].rank(ascending=True)
dw2['RankD'] = dw2.groupby(['Season'])['MeanD'].rank(ascending=True)
dw2['RankG'] = dw2.groupby(['Season'])['MeanG'].rank(ascending=True)
dw2 = dw2.sort_values(['Season', 'RankWin', 'RankC', 'RankW', 'RankD', 'RankD'], ascending=[True, True, True, True, True, True])
dw2.head(30)

,Season,TeamCode,GP,W,L,GF,GA,WinPc,LossPc,MeanC,MeanW,MeanD,MeanG,RankWin,RankC,RankW,RankD,RankG
180,2010,VAN,82,54,28,268,190,0.658537,0.341463,2.121951,1.934146,1.816841,1.000000,1.0,14.0,8.0,10.0,1.5
12,2010,PIT,82,49,33,244,203,0.597561,0.402439,2.103600,2.246051,1.926713,1.756098,2.0,11.0,17.0,18.0,12.0
44,2010,SJ,82,48,34,253,216,0.585366,0.414634,1.477439,2.184727,1.750639,1.560976,3.5,1.0,16.0,8.0,10.0
84,2010,WSH,82,48,34,230,203,0.585366,0.414634,2.563444,1.893815,1.920732,1.536585,3.5,27.0,6.0,17.0,9.0
68,2010,DET,82,47,35,263,241,0.573171,0.426829,1.575145,2.053252,1.661150,2.121951,6.0,2.0,13.0,2.0,21.0
8,2010,PHI,82,47,35,266,233,0.573171,0.426829,1.803659,1.982578,1.613821,1.987805,6.0,6.0,9.0,1.0,17.0
64,2010,ANA,82,47,35,241,237,0.573171,0.426829,2.454268,2.165258,2.104820,1.682927,6.0,26.0,15.0,21.0,11.0
88,2010,BOS,82,46,36,250,200,0.560976,0.439024,1.730778,1.532201,1.835366,1.329268,9.0,5.0,1.0,12.0,7.5
140,2010,TB,82,46,36,252,246,0.560976,0.439024,2.041057,1.826132,1.874274,2.310976,9.0,10.0,3.0,15.0,25.0
176,2010,LA,82,46,36,227,207,0.560976,0.439024,2.345122,2.331252,1.688444,2.000000,9.0,23.0,20.0,3.0,19.5


In [61]:
dw2.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/full_season_team_roster_ranking_all_nhl_positions.csv', index='False', sep=',')
#dw2.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/full_season_team_roster_ranking_all_nhl_positions.csv', index='False', sep=',')

### keep games that have 12 forwards and 6 defensemen per team!!

In [203]:
du = dw.copy()
du.shape

(46916, 18)

In [204]:
du['FCount'] = du['CCount']+ dwfd['WCount']
du.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank,GF,GA,GD,WinTeam,LossTeam,RosterCount,PositionCount,CCount,WCount,DCount,GCount,FCount
0,2010,20001,MTL,11.0,GOMEZ,C,2.0,2,3,1,TOR,MTL,19.0,7.0,7.0,5.0,6.0,1.0,12.0
1,2010,20001,MTL,21.0,GIONTA,W,2.0,2,3,1,TOR,MTL,19.0,5.0,7.0,5.0,6.0,1.0,12.0
2,2010,20001,MTL,57.0,POULIOT,W,2.0,2,3,1,TOR,MTL,19.0,5.0,7.0,5.0,6.0,1.0,12.0
3,2010,20001,MTL,26.0,GORGES,D,2.0,2,3,1,TOR,MTL,19.0,6.0,7.0,5.0,6.0,1.0,12.0
4,2010,20001,MTL,75.0,GILL,D,2.0,2,3,1,TOR,MTL,19.0,6.0,7.0,5.0,6.0,1.0,12.0


In [70]:
du = du.groupby(['Season', 'GameNumber']).filter(lambda x: ((x['FCount'] == 12) & (x['DCount'] == 6)).all())
du.shape

(34917, 19)

In [205]:
du.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/12f_6d_g_game_team_player_ranking_nhl_positions.csv', index='False', sep=',')
#du.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/12f_6d__g_game_team_player_ranking_nhl_positions.csv', index='False', sep=',')

### keep games that have 4 C, 8 W, 6 D and 1G per team!!!!

In [206]:
dy = dw.copy()
dy.shape

(46916, 18)

In [207]:
dy = dy.groupby(['Season', 'GameNumber']).filter(lambda x: ((x['CCount'] == 4) & (x['WCount'] == 8) & (x['DCount'] == 6) & (x['GCount'] == 1)).all())
dy.shape

(1406, 18)

In [208]:
dy.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/4c_8w_6d_1g_game_team_player_ranking_nhl_positions.csv', index='False', sep=',')
#dy.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/4c_8w_6d__1g_game_team_player_ranking_nhl_positions.csv', index='False', sep=',')

- create a new dataset using team roster player rank

In [209]:
dz = dy.copy()

In [210]:
dz = dz.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerPosition', 'GF', 'GA', 'RosterCount', 'PositionCount', 'CCount', 'WCount', 'DCount', 'GCount', 'WinTeam', 'LossTeam'], as_index=False)['Rank'].mean()
dz.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,GCount,WinTeam,LossTeam,Rank
0,2010,20023,LA,C,1,1,19.0,4.0,4.0,8.0,6.0,1.0,LA,VAN,2.500000
1,2010,20023,LA,D,1,1,19.0,6.0,4.0,8.0,6.0,1.0,LA,VAN,1.833333
2,2010,20023,LA,G,1,1,19.0,1.0,4.0,8.0,6.0,1.0,LA,VAN,2.000000
3,2010,20023,LA,W,1,1,19.0,8.0,4.0,8.0,6.0,1.0,LA,VAN,2.375000
4,2010,20023,VAN,C,1,1,19.0,4.0,4.0,8.0,6.0,1.0,LA,VAN,2.250000


In [211]:
dz.shape

(296, 15)

- create columns for team win and team loss. 

In [212]:
dz['TeamWin'] =  dz.apply(lambda x: 1 if x['TeamCode']==x['WinTeam'] else 0, 1)
dz['TeamLos'] =  dz.apply(lambda x: 1 if x['TeamCode']==x['LossTeam'] else 0, 1)
dz.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,GCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos
0,2010,20023,LA,C,1,1,19.0,4.0,4.0,8.0,6.0,1.0,LA,VAN,2.500000,1,0
1,2010,20023,LA,D,1,1,19.0,6.0,4.0,8.0,6.0,1.0,LA,VAN,1.833333,1,0
2,2010,20023,LA,G,1,1,19.0,1.0,4.0,8.0,6.0,1.0,LA,VAN,2.000000,1,0
3,2010,20023,LA,W,1,1,19.0,8.0,4.0,8.0,6.0,1.0,LA,VAN,2.375000,1,0
4,2010,20023,VAN,C,1,1,19.0,4.0,4.0,8.0,6.0,1.0,LA,VAN,2.250000,0,1


- display games played, games won, games loss, goals for and goals against by team for the season.

In [213]:
dz['GP'] = dz.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GameNumber'].transform('count')
dz['GW'] = dz.groupby(['Season', 'PlayerPosition', 'TeamCode'])['TeamWin'].transform('sum')
dz['GL'] = dz.groupby(['Season', 'PlayerPosition', 'TeamCode'])['TeamLos'].transform('sum')
dz['GF'] = dz.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GF'].transform('sum')
dz['GA'] = dz.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GA'].transform('sum')
dz.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,GCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL
0,2010,20023,LA,C,16,22,19.0,4.0,4.0,8.0,6.0,1.0,LA,VAN,2.500000,1,0,9,4,5
1,2010,20023,LA,D,16,22,19.0,6.0,4.0,8.0,6.0,1.0,LA,VAN,1.833333,1,0,9,4,5
2,2010,20023,LA,G,16,22,19.0,1.0,4.0,8.0,6.0,1.0,LA,VAN,2.000000,1,0,9,4,5
3,2010,20023,LA,W,16,22,19.0,8.0,4.0,8.0,6.0,1.0,LA,VAN,2.375000,1,0,9,4,5
4,2010,20023,VAN,C,17,9,19.0,4.0,4.0,8.0,6.0,1.0,LA,VAN,2.250000,0,1,5,3,2


- create columns with the mean ranking for centers, wingers, defenseman and goaltenders by team per game.

In [214]:
dz['RankC'] = dz.apply(lambda x: x['Rank'] if x['PlayerPosition']=='C' else np.NaN, 1)
dz['RankW'] = dz.apply(lambda x: x['Rank'] if x['PlayerPosition']=='W' else np.NaN, 1)
dz['RankD'] = dz.apply(lambda x: x['Rank'] if x['PlayerPosition']=='D' else np.NaN, 1)
dz['RankG'] = dz.apply(lambda x: x['Rank'] if x['PlayerPosition']=='G' else np.NaN, 1)

dz['RankC'] = dz.groupby(['Season','GameNumber', 'TeamCode'])['RankC'].apply(lambda x: x.ffill().bfill())
dz['RankW'] = dz.groupby(['Season','GameNumber', 'TeamCode'])['RankW'].apply(lambda x: x.ffill().bfill())
dz['RankD'] = dz.groupby(['Season','GameNumber', 'TeamCode'])['RankD'].apply(lambda x: x.ffill().bfill())
dz['RankG'] = dz.groupby(['Season','GameNumber', 'TeamCode'])['RankG'].apply(lambda x: x.ffill().bfill())
dz.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,GCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,RankC,RankW,RankD,RankG
0,2010,20023,LA,C,16,22,19.0,4.0,4.0,8.0,6.0,1.0,LA,VAN,2.500000,1,0,9,4,5,2.50,2.375,1.833333,2.0
1,2010,20023,LA,D,16,22,19.0,6.0,4.0,8.0,6.0,1.0,LA,VAN,1.833333,1,0,9,4,5,2.50,2.375,1.833333,2.0
2,2010,20023,LA,G,16,22,19.0,1.0,4.0,8.0,6.0,1.0,LA,VAN,2.000000,1,0,9,4,5,2.50,2.375,1.833333,2.0
3,2010,20023,LA,W,16,22,19.0,8.0,4.0,8.0,6.0,1.0,LA,VAN,2.375000,1,0,9,4,5,2.50,2.375,1.833333,2.0
4,2010,20023,VAN,C,17,9,19.0,4.0,4.0,8.0,6.0,1.0,LA,VAN,2.250000,0,1,5,3,2,2.25,2.250,1.666667,1.0


- compute the mean per position by team for the season.

In [215]:
dz['MeanC']= dz.groupby(['Season', 'TeamCode'])['RankC'].transform('mean')
dz['MeanW']= dz.groupby(['Season', 'TeamCode'])['RankW'].transform('mean')
dz['MeanD']= dz.groupby(['Season', 'TeamCode'])['RankD'].transform('mean')
dz['MeanG']= dz.groupby(['Season', 'TeamCode'])['RankG'].transform('mean')
dz.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,GCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,RankC,RankW,RankD,RankG,MeanC,MeanW,MeanD,MeanG
0,2010,20023,LA,C,16,22,19.0,4.0,4.0,8.0,6.0,1.0,LA,VAN,2.500000,1,0,9,4,5,2.50,2.375,1.833333,2.0,2.333333,2.361111,1.703704,2.0
1,2010,20023,LA,D,16,22,19.0,6.0,4.0,8.0,6.0,1.0,LA,VAN,1.833333,1,0,9,4,5,2.50,2.375,1.833333,2.0,2.333333,2.361111,1.703704,2.0
2,2010,20023,LA,G,16,22,19.0,1.0,4.0,8.0,6.0,1.0,LA,VAN,2.000000,1,0,9,4,5,2.50,2.375,1.833333,2.0,2.333333,2.361111,1.703704,2.0
3,2010,20023,LA,W,16,22,19.0,8.0,4.0,8.0,6.0,1.0,LA,VAN,2.375000,1,0,9,4,5,2.50,2.375,1.833333,2.0,2.333333,2.361111,1.703704,2.0
4,2010,20023,VAN,C,17,9,19.0,4.0,4.0,8.0,6.0,1.0,LA,VAN,2.250000,0,1,5,3,2,2.25,2.250,1.666667,1.0,2.250000,1.950000,1.666667,1.0


- display the quantity of wins and losses per team ( roster of 4 centers, 8 wingers, 6 defensemen and a goaltender)

In [216]:
dz['L'] = dz.apply(lambda x: x['GL'] if x['TeamCode']== x['LossTeam'] else (x['GP'] - x['GW']), 1)
dz['W'] = dz.apply(lambda x: x['GW'] if x['TeamCode']== x['WinTeam'] else (x['GP'] - x['GL']), 1)
dz.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,GCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,RankC,RankW,RankD,RankG,MeanC,MeanW,MeanD,MeanG,L,W
0,2010,20023,LA,C,16,22,19.0,4.0,4.0,8.0,6.0,1.0,LA,VAN,2.500000,1,0,9,4,5,2.50,2.375,1.833333,2.0,2.333333,2.361111,1.703704,2.0,5,4
1,2010,20023,LA,D,16,22,19.0,6.0,4.0,8.0,6.0,1.0,LA,VAN,1.833333,1,0,9,4,5,2.50,2.375,1.833333,2.0,2.333333,2.361111,1.703704,2.0,5,4
2,2010,20023,LA,G,16,22,19.0,1.0,4.0,8.0,6.0,1.0,LA,VAN,2.000000,1,0,9,4,5,2.50,2.375,1.833333,2.0,2.333333,2.361111,1.703704,2.0,5,4
3,2010,20023,LA,W,16,22,19.0,8.0,4.0,8.0,6.0,1.0,LA,VAN,2.375000,1,0,9,4,5,2.50,2.375,1.833333,2.0,2.333333,2.361111,1.703704,2.0,5,4
4,2010,20023,VAN,C,17,9,19.0,4.0,4.0,8.0,6.0,1.0,LA,VAN,2.250000,0,1,5,3,2,2.25,2.250,1.666667,1.0,2.250000,1.950000,1.666667,1.0,2,3


- compute win and loss percent by team. Drop duplicate observations.

In [217]:
dz = dz[['Season', 'TeamCode', 'GP', 'L', 'W', 'GF', 'GA', 'MeanC', 'MeanW', 'MeanD', 'MeanG']]
dz = dz.drop_duplicates(['Season', 'TeamCode'])
dz['WinPc'] = dz['W']/ dz['GP']
dz['LossPc'] = dz['L']/ dz['GP']

dz = dz[['Season', 'TeamCode', 'GP','W', 'L', 'GF', 'GA', 'WinPc', 'LossPc', 'MeanC', 'MeanW', 'MeanD', 'MeanG']]
dz.head()

,Season,TeamCode,GP,W,L,GF,GA,WinPc,LossPc,MeanC,MeanW,MeanD,MeanG
0,2010,LA,9,4,5,16,22,0.444444,0.555556,2.333333,2.361111,1.703704,2.0
4,2010,VAN,5,3,2,17,9,0.600000,0.400000,2.250000,1.950000,1.666667,1.0
8,2010,CGY,5,2,3,13,13,0.400000,0.600000,1.600000,1.925000,1.900000,2.0
16,2010,NYI,5,0,5,10,20,0.000000,1.000000,2.200000,2.600000,2.100000,2.2
20,2010,WSH,1,1,0,2,1,1.000000,0.000000,2.750000,1.875000,2.000000,2.0


- rank teams based on win percent, mean centres, mean wingers, mean defensemen and mean goaltenders. 

In [218]:
dz['RankWin'] = dz.groupby(['Season'])['WinPc'].rank(ascending=False)
dz['RankC'] = dz.groupby(['Season'])['MeanC'].rank(ascending=True)
dz['RankW'] = dz.groupby(['Season'])['MeanW'].rank(ascending=True)
dz['RankD'] = dz.groupby(['Season'])['MeanD'].rank(ascending=True)
dz['RankG'] = dz.groupby(['Season'])['MeanG'].rank(ascending=True)
dz = dz.sort_values(['Season', 'RankWin', 'RankC', 'RankW', 'RankD', 'RankG'], ascending=[True, True, True, True, True, True])
dz.head(30)

,Season,TeamCode,GP,W,L,GF,GA,WinPc,LossPc,MeanC,MeanW,MeanD,MeanG,RankWin,RankC,RankW,RankD,RankG
144,2010,EDM,1,1,0,4,3,1.000000,0.000000,2.250000,2.750000,2.500000,2.000000,1.5,10.5,16.5,17.0,11.0
20,2010,WSH,1,1,0,2,1,1.000000,0.000000,2.750000,1.875000,2.000000,2.000000,1.5,17.0,2.5,11.5,11.0
24,2010,ANA,8,7,1,26,17,0.875000,0.125000,2.156250,2.250000,2.041667,1.500000,3.0,7.0,8.0,13.0,4.5
152,2010,CAR,5,4,1,15,13,0.800000,0.200000,1.750000,2.375000,1.933333,1.400000,5.0,3.0,10.0,9.0,3.0
36,2010,ATL,5,4,1,19,17,0.800000,0.200000,2.050000,2.500000,1.833333,2.600000,5.0,6.0,12.0,7.0,17.0
164,2010,OTT,5,4,1,21,9,0.800000,0.200000,2.500000,2.725000,2.466667,1.800000,5.0,15.5,15.0,16.0,7.0
196,2010,CHI,3,2,1,10,4,0.666667,0.333333,2.500000,1.791667,1.500000,2.000000,7.0,15.5,1.0,1.0,11.0
4,2010,VAN,5,3,2,17,9,0.600000,0.400000,2.250000,1.950000,1.666667,1.000000,8.0,10.5,5.0,2.0,1.5
0,2010,LA,9,4,5,16,22,0.444444,0.555556,2.333333,2.361111,1.703704,2.000000,9.0,14.0,9.0,3.0,11.0
96,2010,STL,7,3,4,16,20,0.428571,0.571429,1.642857,2.410714,1.738095,2.142857,10.0,2.0,11.0,5.0,15.0


In [219]:
dz.shape

(17, 18)

In [220]:
dz.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/season_team_roster_ranking_4C_8W_6D_1G_nhl_positions.csv', index='False', sep=',')
#dz.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/season_team_roster_ranking_4C_8W_6D_1G_nhl_positions.csv', index='False', sep=',')
